In [ ]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from rasterio.plot import show
import rasterio
from shapely.geometry import LineString
from scipy.stats import gaussian_kde

In [ ]:
# Load Finland boundary (projected CRS, e.g., EPSG:3067)
finland = gpd.read_file("finland_boundary.shp").to_crs(epsg=3067)

# Load stream network (ensure it's clipped to Finland and in the same CRS)
streams = gpd.read_file("rivers_finland.shp").to_crs(finland.crs)

# Load DEM (optional, for hillshade)
dem = rasterio.open("finland_dem.tif")

In [ ]:
def line_to_points(line, spacing=100):  # 100m spacing
    points = []
    for geom in line.geometry:
        if isinstance(geom, LineString):
            length = geom.length
            for distance in np.arange(0, length, spacing):
                point = geom.interpolate(distance)
                points.append([point.x, point.y])
    return np.array(points)

# Generate points along all streams
stream_points = line_to_points(streams, spacing=100)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

# Assume 'finland' is your GeoDataFrame reprojected to EPSG:3067.
xmin, ymin, xmax, ymax = finland.total_bounds  # e.g. [75242.32, 6637484.63, 732058.46, 7774506.09]
grid_res = 4891  # resolution in meters
xi = np.arange(xmin, xmax, grid_res)
yi = np.arange(ymin, ymax, grid_res)
print("xi shape:", xi.shape)  # Expected around (135,)
print("yi shape:", yi.shape)  # Expected around (233,)

xx, yy = np.meshgrid(xi, yi)
n_rows, n_cols = xx.shape
print("Grid shape (rows x cols):", n_rows, "x", n_cols)



In [ ]:
# stream_points should be an array of shape (n_points, 2)
kde_cpu = gaussian_kde(stream_points.T, bw_method=0.1)
print("CPU KDE object created.")


In [ ]:
# Partition the grid along rows.
chunk_size = 10  # adjust as needed (number of rows per chunk)
chunks = []
for start in range(0, n_rows, chunk_size):
    end = min(start + chunk_size, n_rows)
    # Each chunk is built from the rows [start:end] of the grid.
    # We stack the x and y coordinates to form a (2, m) array.
    chunk_grid = np.vstack([xx[start:end].ravel(), yy[start:end].ravel()])
    chunks.append((start, end, chunk_grid))
print(f"Total chunks created: {len(chunks)}")

In [ ]:
# Define a worker function that initializes (or reuses) a cached KDE object on the worker.
def process_cpu_with_init(chunk_grid, bw_method=0.1):
    global _cpu_kde
    try:
        _cpu_kde
    except NameError:
        from scipy.stats import gaussian_kde
        # Initialize using stream_points.T (shape (2, n))
        _cpu_kde = gaussian_kde(stream_points.T, bw_method=bw_method)
    # Evaluate KDE on the given chunk_grid (shape (2, m))
    return _cpu_kde(chunk_grid)

# Set up a Dask local cluster for CPU processing.
from dask.distributed import LocalCluster, Client, progress
cluster = LocalCluster(n_workers=4, threads_per_worker=1)  # adjust number of workers as needed
client = Client(cluster)
print("Cluster workers:", client.scheduler_info()["workers"])


In [ ]:
# Submit each chunk as a separate task.
futures = []
for idx, (start, end, chunk_grid) in enumerate(chunks):
    fut = client.submit(process_cpu_with_init, chunk_grid, bw_method=0.1)
    futures.append((start, end, fut))
print("Submitted", len(futures), "tasks.")
progress([fut for _, _, fut in futures])

In [ ]:
density_full = np.empty((n_rows, n_cols))
for start, end, fut in futures:
    chunk_result = fut.result()  # This is a 1D array for the chunk.
    n_chunk_rows = end - start
    density_full[start:end, :] = chunk_result.reshape(n_chunk_rows, n_cols)
print("Reassembled density grid shape:", density_full.shape)

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyArrowPatch

fig, ax = plt.subplots(figsize=(10, 10))

# Display the density grid using the 'nipy_spectral' colormap.
im = ax.imshow(density_full, extent=[xmin, xmax, ymin, ymax],
               origin='lower', cmap='nipy_spectral', alpha=0.8)

# Overlay Finland boundaries from the shapefile.
finland.boundary.plot(ax=ax, edgecolor='black', linewidth=1.5)

cbar = fig.colorbar(im, ax=ax, fraction=0.036, pad=0.04)
cbar.set_label("Density (km/km²)", fontsize=12)


# Add a scale bar.
scalebar_length = 100000  # 100 km
x0, x1 = ax.get_xlim()
y0, y1 = ax.get_ylim()
scalebar_x = x0 + 0.3*(x1 - x0)
scalebar_y = y0 + 0.1*(y1 - y0)
ax.plot([scalebar_x, scalebar_x + scalebar_length],
        [scalebar_y, scalebar_y], 'k-', lw=3)
ax.text(scalebar_x + scalebar_length/2, scalebar_y - 0.02*(y1-y0),
        f'{scalebar_length/1000:.0f} km', ha='center', va='top', fontsize=12)

# Add a north arrow.
arrow_x = x1 - 0.1*(x1 - x0)
arrow_y = y0 + 0.9*(y1 - y0)
# Add a north arrow with white "N".
ax.annotate('N', 
            xy=(arrow_x, arrow_y + 0.05*(y1-y0)),
            xytext=(arrow_x, arrow_y),
            arrowprops=dict(facecolor='white', width=5, headwidth=15),
            ha='center', va='center', fontsize=14, color='white')

# Title and labels.
ax.set_title("Drainage density Map of Finland", fontsize=16)
ax.set_xlabel("Easting (m)", fontsize=14)
ax.set_ylabel("Northing (m)", fontsize=14)

plt.tight_layout()
plt.show()